In [1]:
from powersimdata.scenario.scenario import Scenario
from powersimdata.design.scenario_info import ScenarioInfo
import json
import os
import pandas as pd

In [2]:
from powersimdata.design.clean_capacity_scaling import CollaborativeStrategyManager,\
IndependentStrategyManager, TargetManager, ResourceManager, Resource

In [3]:
scenario_string = '403'
targets_info_location = os.path.join('data', 'eastern_2030_clean_energy_targets.csv')

Load a scenario to that we will extract resource information from

In [4]:
scenario = Scenario(scenario_string)

SCENARIO: test | EasternBase_2020_3

--> State
analyze


Create a scenario_info object from which we calculate resource properties 

In [5]:
scenario_info = ScenarioInfo(scenario)

--> Loading PG
--> Loading grid
Loading bus
Loading plant
Loading heat_rate_curve
Loading gencost_before
Loading gencost_after
Loading branch
Loading dcline
Loading sub
Loading bus2sub
--> Loading demand
Multiply demand in Maine (#1) by 0.98
Multiply demand in New Hampshire (#2) by 0.98
Multiply demand in Vermont (#3) by 0.98
Multiply demand in Massachusetts (#4) by 0.98
Multiply demand in Rhode Island (#5) by 0.98
Multiply demand in Connecticut (#6) by 0.98
Multiply demand in New York City (#7) by 0.99
Multiply demand in Upstate New York (#8) by 0.99
Multiply demand in New Jersey (#9) by 1.04
Multiply demand in Pennsylvania Eastern (#10) by 1.02
Multiply demand in Pennsylvania Western (#11) by 1.02
Multiply demand in Delaware (#12) by 1.04
Multiply demand in Maryland (#13) by 1.01
Multiply demand in Virginia Mountains (#14) by 1.06
Multiply demand in Virginia Tidewater (#15) by 1.06
Multiply demand in North Carolina (#16) by 1.04
Multiply demand in Western North Carolina (#17) by 1.04

Read in external parameters for region capacity targets

In [6]:
eastern = pd.read_csv(targets_info_location)
eastern

,region_name,2016_demand,2030_2016_demand_scalar,total_demand,ce_target_fraction,ce_category,allowed_resources,external_ce_historical_amount,solar_percentage
0,Alabama,83239040,1.028367,8.560028e+07,0.00,NaN,"solar,wind",NaN,NaN
1,Arkansas,49463250,1.165510,5.764994e+07,0.00,NaN,"solar,wind",NaN,NaN
2,Connecticut,32931880,0.945433,3.113488e+07,0.44,Renewables,"geothermal, hydro, solar, wind",NaN,NaN
3,Delaware,12381650,1.165510,1.443094e+07,0.25,Renewables,"geothermal, hydro, solar, wind",NaN,NaN
4,Florida,253533010,1.149474,2.914297e+08,0.00,NaN,"solar,wind",NaN,NaN
5,Georgia,171946010,1.028367,1.768236e+08,0.00,NaN,"solar,wind",NaN,NaN
6,Iowa,39669870,1.216553,4.826049e+07,0.00,NaN,"solar,wind",NaN,NaN
7,Illinois,165066780,1.069337,1.765121e+08,0.25,Renewables,"hydro, solar, wind",NaN,NaN
8,Indiana,96674130,1.173606,1.134574e+08,0.10,Clean,"geothermal, hydro, nuclear, solar, wind",NaN,NaN
9,Kansas,51402860,1.087356,5.589320e+07,0.20,Renewables,"hydro, solar, wind",NaN,NaN


Clean up the missing data with appropriate defaults

In [7]:
eastern['external_ce_historical_amount'] = eastern['external_ce_historical_amount'].fillna(0)
eastern['allowed_resources'] = eastern['allowed_resources'].fillna('solar,wind')
eastern['ce_category'] = eastern['ce_category'].fillna('TBD')
eastern['solar_percentage'] = eastern['solar_percentage'].fillna('None')
eastern

,region_name,2016_demand,2030_2016_demand_scalar,total_demand,ce_target_fraction,ce_category,allowed_resources,external_ce_historical_amount,solar_percentage
0,Alabama,83239040,1.028367,8.560028e+07,0.00,TBD,"solar,wind",0.0,None
1,Arkansas,49463250,1.165510,5.764994e+07,0.00,TBD,"solar,wind",0.0,None
2,Connecticut,32931880,0.945433,3.113488e+07,0.44,Renewables,"geothermal, hydro, solar, wind",0.0,None
3,Delaware,12381650,1.165510,1.443094e+07,0.25,Renewables,"geothermal, hydro, solar, wind",0.0,None
4,Florida,253533010,1.149474,2.914297e+08,0.00,TBD,"solar,wind",0.0,None
5,Georgia,171946010,1.028367,1.768236e+08,0.00,TBD,"solar,wind",0.0,None
6,Iowa,39669870,1.216553,4.826049e+07,0.00,TBD,"solar,wind",0.0,None
7,Illinois,165066780,1.069337,1.765121e+08,0.25,Renewables,"hydro, solar, wind",0.0,None
8,Indiana,96674130,1.173606,1.134574e+08,0.10,Clean,"geothermal, hydro, nuclear, solar, wind",0.0,None
9,Kansas,51402860,1.087356,5.589320e+07,0.20,Renewables,"hydro, solar, wind",0.0,None


Create Independent and Collaborative Strategy objects and set the simulation hours to be a full year

In [8]:
independent_strategy_manager = IndependentStrategyManager()
collaborative_strategy_manager = CollaborativeStrategyManager()
#AbstractStrategyManager.set_next_sim_hours(8784)

Populate strategy managers with targets created from external data

In [9]:
independent_strategy_manager.targets_from_data_frame(eastern)
collaborative_strategy_manager.targets_from_data_frame(eastern)

For each target region, add the available resources and also create allowed resources that are not currently present

In [10]:
# define start and end times of the simulation
start_time = '2016-01-01 00:00:00'
end_time = '2016-12-31 23:00:00'

# add resource objects to targets
independent_strategy_manager.populate_targets_with_resources(scenario_info, start_time, end_time)


Alabama

Invalid resource type
Invalid resource type
Added resource ng!

Added resource wind!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource other!

Added resource hydro!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!

Invalid resource type
Invalid resource type
Added resource coal!


Arkansas

Invalid resource type
Invalid resource type
Added resource ng!

Added resource wind!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource other!

Added resource hydro!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!

Invalid resource type
Invalid resource type
Added resource coal!


Connecticut

Invalid resource type
Invalid resource type
Added resource ng!

Added resource wind!

No existing resource geothermal!
Invalid resource type
No such type of generator 

/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/scenario_info.py:118: UserWarning: No such type of generator in the area specified!
  warnings.warn('No such type of generator in the area specified!')


Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!

Invalid resource type
Invalid resource type
Added resource coal!


Delaware

Invalid resource type
Invalid resource type
Added resource ng!

Added resource wind!

No existing resource geothermal!
Invalid resource type
No such type of generator in the area specified. Division by zero.
Added resource geothermal!

Invalid resource type
Invalid resource type
Added resource other!

No existing resource hydro!
No such type of generator in the area specified. Division by zero.
Added resource hydro!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!

Invalid resource type
Invalid resource type
Added resource coal!


Florida

Invalid resource type
Invalid resource type
Added resource ng!

Added resource wind!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource other!

Added resource hydro!

Added

In [11]:
collaborative_strategy_manager.populate_targets_with_resources(scenario_info, start_time, end_time)


Alabama

Invalid resource type
Invalid resource type
Added resource ng!

Added resource wind!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource other!

Added resource hydro!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!

Invalid resource type
Invalid resource type
Added resource coal!


Arkansas

Invalid resource type
Invalid resource type
Added resource ng!

Added resource wind!

Invalid resource type
Invalid resource type
Added resource nuclear!

Invalid resource type
Invalid resource type
Added resource other!

Added resource hydro!

Added resource solar!

Invalid resource type
Invalid resource type
Added resource dfo!

Invalid resource type
Invalid resource type
Added resource coal!


Connecticut

Invalid resource type
Invalid resource type
Added resource ng!

Added resource wind!

No existing resource geothermal!
Invalid resource type
No such type of generator 

Calculate the Independent Strategy next capacities along with intermediate values

In [12]:
independent_next_capacities = independent_strategy_manager.data_frame_of_next_capacities()
independent_next_capacities

,ce_target_fraction,ce_target,previous_ce_generation,clean_energy_shortfall,solar_added_capacity,wind_added_capacity,solar_prev_capacity,wind_prev_capacity,solar_expected_cap_factor,wind_expected_cap_factor,next_solar_capacity,next_wind_capacity
region_name,,,,,,,,,,,,
Alabama,0.00,0.000000e+00,4.425426e+05,0.000000e+00,0.000000,0.000000,196.9,1.0,0.2551,0.1563,196.900000,1.000000
Arkansas,0.00,0.000000e+00,2.438784e+05,0.000000e+00,0.000000,0.000000,110.0,1.0,0.2499,0.2733,110.000000,1.000000
Connecticut,0.44,1.369935e+07,7.333156e+05,1.296603e+07,6821.197144,258.378680,132.0,5.0,0.2073,0.2402,6953.197144,263.378680
Delaware,0.25,3.607736e+06,7.630009e+04,3.531436e+06,1740.231326,92.565496,37.6,2.0,0.2180,0.2448,1777.831326,94.565496
Florida,0.00,0.000000e+00,4.089562e+06,0.000000e+00,0.000000,0.000000,2143.4,3.0,0.2170,0.1619,2143.400000,3.000000
Georgia,0.00,0.000000e+00,3.195848e+06,0.000000e+00,0.000000,0.000000,1516.8,2.0,0.2397,0.1077,1516.800000,2.000000
Iowa,0.00,0.000000e+00,2.969101e+07,0.000000e+00,0.000000,0.000000,12.3,9905.8,0.1919,0.3410,12.300000,9905.800000
Illinois,0.25,4.412802e+07,1.409325e+07,3.003477e+07,82.055444,11203.690983,38.1,5202.1,0.2035,0.3037,120.155444,16405.790983
Indiana,0.10,1.134574e+07,6.706850e+06,4.638887e+06,180.639121,1705.806387,244.6,2309.8,0.1973,0.2887,425.239121,4015.606387


Calculate the Collaborative Strategy next capacities along with intermediate values

In [13]:
collaborative_next_capacities = collaborative_strategy_manager.data_frame_of_next_capacities()
collaborative_next_capacities

,ce_target_fraction,ce_target,previous_ce_generation,clean_energy_shortfall,solar_scaling,wind_scaling,solar_prev_capacity,wind_prev_capacity,next_solar_capacity,next_wind_capacity
region_name,,,,,,,,,,
Alabama,0.00,0.000000e+00,4.425426e+05,0.000000e+00,2.233972,2.233972,196.9,1.0,439.869002,2.233972
Arkansas,0.00,0.000000e+00,2.438784e+05,0.000000e+00,2.233972,2.233972,110.0,1.0,245.736872,2.233972
Connecticut,0.44,1.369935e+07,7.333156e+05,1.296603e+07,2.233972,2.233972,132.0,5.0,294.884247,11.169858
Delaware,0.25,3.607736e+06,7.630009e+04,3.531436e+06,2.233972,2.233972,37.6,2.0,83.997331,4.467943
Florida,0.00,0.000000e+00,4.089562e+06,0.000000e+00,2.233972,2.233972,2143.4,3.0,4788.294659,6.701915
Georgia,0.00,0.000000e+00,3.195848e+06,0.000000e+00,2.233972,2.233972,1516.8,2.0,3388.488074,4.467943
Iowa,0.00,0.000000e+00,2.969101e+07,0.000000e+00,2.233972,2.233972,12.3,9905.8,27.477850,22129.275558
Illinois,0.25,4.412802e+07,1.409325e+07,3.003477e+07,2.233972,2.233972,38.1,5202.1,85.114317,11621.343494
Indiana,0.10,1.134574e+07,6.706850e+06,4.638887e+06,2.233972,2.233972,244.6,2309.8,546.429446,5160.027528


Export all target properties as rows: first, use jsonpickle to convert targets to json,
and second, use json_normalize to flatten this json hierarchy into a dataframe

Output dataframe of next capacities

In [14]:
cap_planning_df = independent_strategy_manager.output_targets_dataframe()
cap_planning_df

,allowed_resources,ce_category,ce_target,ce_target_fraction,external_ce_historical_amount,solar_percentage,total_demand,resources.resources.coal.addl_curtailment,resources.resources.coal.name,resources.resources.coal.no_congestion_cap_factor,...,resources.resources.wind.prev_generation,resources.resources.wind.prev_scenario_num,resources.resources.geothermal.addl_curtailment,resources.resources.geothermal.name,resources.resources.geothermal.no_congestion_cap_factor,resources.resources.geothermal.prev_cap_factor,resources.resources.geothermal.prev_capacity,resources.resources.geothermal.prev_curtailment,resources.resources.geothermal.prev_generation,resources.resources.geothermal.prev_scenario_num
region_name,,,,,,,,,,,,,,,,,,,,,
Alabama,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,8.560028e+07,0.0,coal,0.0,...,1.372931e+03,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Arkansas,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,5.764994e+07,0.0,coal,0.0,...,2.400567e+03,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Connecticut,"[geothermal, hydro, solar, wind]",Renewables,1.369935e+07,0.44,0.0,None,3.113488e+07,0.0,coal,0.0,...,1.054965e+04,403,0.0,geothermal,0.0,0.0,0.0,0.0,0.0,403.0
Delaware,"[geothermal, hydro, solar, wind]",Renewables,3.607736e+06,0.25,0.0,None,1.443094e+07,0.0,coal,0.0,...,4.300648e+03,403,0.0,geothermal,0.0,0.0,0.0,0.0,0.0,403.0
Florida,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,2.914297e+08,0.0,coal,0.0,...,4.266977e+03,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Georgia,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,1.768236e+08,0.0,coal,0.0,...,1.891935e+03,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Iowa,"[solar, wind]",TBD,0.000000e+00,0.00,0.0,None,4.826049e+07,0.0,coal,0.0,...,2.967028e+07,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Illinois,"[hydro, solar, wind]",Renewables,4.412802e+07,0.25,0.0,None,1.765121e+08,0.0,coal,0.0,...,1.387634e+07,403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Indiana,"[geothermal, hydro, nuclear, solar, wind]",Clean,1.134574e+07,0.10,0.0,None,1.134574e+08,0.0,coal,0.0,...,5.856664e+06,403,0.0,geothermal,0.0,0.0,0.0,0.0,0.0,403.0


Output a table of capacities for all target regions and resources

In [15]:
gen_capacity = independent_strategy_manager.output_capacities_table()
gen_capacity

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
'wind_offshore'
Resource wind_offshore not found in target region Alabama
'wind_offshore'
Resource wind_offshore not found in target region Arkansas
'wind_offshore'
Resource wind_offshore not found in target region Connecticut
'nuclear'
Resource nuclear not found in target region Delaware
'wind_offshore'
Resource wind_offshore not found in target region Delaware
'wind_offshore'
Resource wind_offshore not found in target region Florida
'wind_offshore'
Resource wind_offshore not found in target region Georgia
'wind_offshore'
Resource wind_offshore not found in target region Iowa
'wind_offshore'
Resource wind_offshore not found in target region Illinois
'wind_offshore'
Resource wind_offshore not found in target region Indiana
'wind_offshore'
Resource wind_offshore not found in target region Kansas
'nuclear'
Resource nuclear not found in target re

,ng,dfo,hydro,wind,coal,nuclear,solar,other,wind_offshore
Alabama,15433,47.8,3319.3,1,5504.5,4547.11,196.9,381.002,0
Arkansas,7229.4,12.3,1348.9,1,5487,1529.42,110,156.97,0
Connecticut,5331.5,2816.8,149.5,263.379,400,1888.9,6953.2,166,0
Delaware,2978.6,121.6,0,94.5655,445.5,0,1777.83,10.3261,0
Florida,46472.7,3794.3,55.7,3,8768,3341.23,2143.4,887,0
Georgia,18652.3,1072.6,3597.8,2,8979.1,3929.32,1516.8,546.008,0
Iowa,3724.8,1086.1,129.2,9905.8,6034.5,535.903,12.3,34.4761,0
Illinois,17600,811,39.7,16405.8,13422,11237,120.155,84.3488,0
Indiana,9767.7,115.5,92.1,4015.61,16981.9,0,425.239,91,0
Kansas,4477.3,648.9,7,6146.3,4864.7,939.697,10,6.69775,0


Output a nested dictionary of scaling factors (other than 1.0) by resource and loadzone name

In [16]:
scale_factor_table = independent_strategy_manager.create_scale_factor_table()
scale_factor_table

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
'wind_offshore'
Resource wind_offshore not found in target region Alabama
'wind_offshore'
Resource wind_offshore not found in target region Arkansas
'wind_offshore'
Resource wind_offshore not found in target region Connecticut
'nuclear'
Resource nuclear not found in target region Delaware
'wind_offshore'
Resource wind_offshore not found in target region Delaware
'wind_offshore'
Resource wind_offshore not found in target region Florida
'wind_offshore'
Resource wind_offshore not found in target region Georgia
'wind_offshore'
Resource wind_offshore not found in target region Iowa
'wind_offshore'
Resource wind_offshore not found in target region Illinois
'wind_offshore'
Resource wind_offshore not found in target region Indiana
'wind_offshore'
Resource wind_offshore not found in target region Kansas
'nuclear'
Resource nuclear not found in target re

/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Florida and resource hydro when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Florida and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Georgia and resource wind when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Georgia and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))


Base grid capacity is zero for loadzone Georgia North and resource wind_offshore
Base grid capacity is zero for loadzone Iowa and resource wind_offshore
Base grid capacity is zero for loadzone Chicago North Illinois and resource wind_offshore
Base grid capacity is zero for loadzone Illinois Downstate and resource wind_offshore
Base grid capacity is zero for loadzone Indiana and resource nuclear
Base grid capacity is zero for loadzone Indiana and resource wind_offshore


/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Illinois and resource hydro when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))


Base grid capacity is zero for loadzone Kansas and resource wind_offshore
Base grid capacity is zero for loadzone Kentucky and resource nuclear
Base grid capacity is zero for loadzone Kentucky and resource wind_offshore
Base grid capacity is zero for loadzone Louisiana and resource wind_offshore
Base grid capacity is zero for loadzone Maine and resource coal
Base grid capacity is zero for loadzone Maine and resource nuclear


...

/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Ohio and resource hydro when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Ohio and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))


Base grid capacity is zero for loadzone Pennsylvania Western and resource wind_offshore
Base grid capacity is zero for loadzone Pennsylvania Eastern and resource wind_offshore
Base grid capacity is zero for loadzone Rhode Island and resource coal
Base grid capacity is zero for loadzone Rhode Island and resource nuclear
Base grid capacity is zero for loadzone South Dakota and resource nuclear
Base grid capacity is zero for loadzone South Dakota and resource wind_offshore
Base grid capacity is zero for loadzone Tennessee and resource wind_offshore
Base grid capacity is zero for loadzone Texas Panhandle and resource dfo
Base grid capacity is zero for loadzone East Texas and resource dfo


/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Texas and resource hydro when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Virginia and resource nuclear when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))
/home/dmuldrew/.local/lib/python3.6/site-packages/powersimdata/design/clean_capacity_scaling.py:172: UserWarning: Attempting to scale target area Virginia and resource solar when base grid capacity is zero!
  'capacity is zero!'.format(tar, res))


Base grid capacity is zero for loadzone Texas Panhandle and resource nuclear
Base grid capacity is zero for loadzone East Texas and resource nuclear
Base grid capacity is zero for loadzone Texas Panhandle and resource wind_offshore
Base grid capacity is zero for loadzone East Texas and resource wind_offshore
Base grid capacity is zero for loadzone Virginia Mountains and resource wind_offshore
Base grid capacity is zero for loadzone Vermont and resource ng
Base grid capacity is zero for loadzone Vermont and resource coal
Base grid capacity is zero for loadzone Vermont and resource nuclear
Base grid capacity is zero for loadzone Vermont and resource wind_offshore
Base grid capacity is zero for loadzone Wisconsin and resource wind_offshore
Base grid capacity is zero for loadzone West Virginia and resource nuclear
Base grid capacity is zero for loadzone West Virginia and resource wind_offshore


defaultdict(dict,
            {'ng': {'Alabama': 1.0032506021583836,
              'Connecticut': 1.0204997663847009,
              'Delaware': 0.9875339831576154,
              'Florida North': 0.9684479161689177,
              'Florida South': 0.9684479161689177,
              'Florida Panhandle': 0.9684479161689177,
              'Iowa': 0.8880397157452772,
              'Indiana': 1.0044732724056193,
              'Kansas': 0.8614981906286743,
              'Louisiana': 0.9337869132555634,
              'Massachusetts': 1.1114740765842086,
              'Maryland': 1.0057792777158723,
              'Maine': 0.9453997786369908,
              'Michigan Southern': 1.016367574982658,
              'Michigan Northern': 1.016367574982658,
              'Minnesota Northern': 1.0750730973302938,
              'Minnesota Southern': 1.0750730973302938,
              'Missouri East': 0.9978717263692048,
              'Missouri West': 0.9978717263692048,
              'Mississippi': 0.91232173

Output a change table of scaling factors other than 1.0

In [17]:
change_table = independent_strategy_manager.create_change_table()
change_table.ct

Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
Reading bus.csv
Reading plant.csv
Reading gencost.csv
Reading branch.csv
Reading dcline.csv
Reading sub.csv
Reading bus2sub.csv
Reading zone.csv
'wind_offshore'
Resource wind_offshore not found in target region Alabama
'wind_offshore'
Resource wind_offshore not found in target region Arkansas
'wind_offshore'
Resource wind_offshore not found in target region Connecticut
'nuclear'
Resource nuclear not found in target region Delaware
'wind_offshore'
Resource wind_offshore not found in target region Delaware
'wind_offshore'
Resource wind_offshore not found in target region Florida
'wind_offshore'
Resource wind_offshore not found in target region Georgia
'wind_offshore'
Resource wind_offshore not found in target region Iowa
'wind_offshore'
Resource wind_offshore not found in target region Illinois
'wind_offshore'
Resource wind_offshore not found in

{'ng': {'zone_id': {24: 1.0032506021583836,
   6: 1.0204997663847009,
   12: 0.9875339831576154,
   22: 0.9684479161689177,
   23: 0.9684479161689177,
   21: 0.9684479161689177,
   39: 0.8880397157452772,
   33: 1.0044732724056193,
   48: 0.8614981906286743,
   43: 0.9337869132555634,
   4: 1.1114740765842086,
   13: 1.0057792777158723,
   1: 0.9453997786369908,
   32: 1.016367574982658,
   31: 1.016367574982658,
   37: 1.0750730973302938,
   38: 1.0750730973302938,
   40: 0.9978717263692048,
   41: 0.9978717263692048,
   25: 0.9123217303976288,
   17: 1.0320508269010866,
   16: 1.0320508269010866,
   51: 1.5073097215042996,
   49: 0.975573147634249,
   9: 0.9177177526117556,
   46: 0.988831201930389,
   7: 0.9974629266151973,
   8: 0.9974629266151973,
   47: 1.0031298217720679,
   11: 1.0020369445578527,
   10: 1.0020369445578527,
   18: 0.9936210843287235,
   45: 0.994642312338667,
   44: 0.994642312338667,
   14: 0.9633879045948965,
   15: 0.9633879045948965,
   36: 0.99786051839619